In [1]:
#Import all the necessary libraries
import os
import numpy as np
import pandas as pd
from keras.layers import Dense,LSTM,SimpleRNN, Embedding
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns 

Using TensorFlow backend.


In [2]:
#setting working directory 
os.chdir("E:\\INSOFE\\text mining\\phone_data")

In [3]:
#Reading the data
#actual_data=pd.read_excel("Data-for-Classification-20180320.xlsx")
actual_data=pd.read_excel("Data-for-Classification-20180320 - V2.xlsx")
actual_data=pd.DataFrame(actual_data)

In [4]:
actual_data.columns

Index(['Summary (Input', 'DATA (Input)', 'categories (Output)',
       'sub_categories (Output)', 'previous_appointment (Output)', 'SLNO'],
      dtype='object')

In [5]:
actual_data.drop('SLNO',axis=1,inplace=True)

In [6]:
#Changing the column names
col_names=["summary","data","categories","sub_categories","appointment"]
actual_data.columns=col_names

In [7]:
actual_data.head(5)

,summary,data,categories,sub_categories,appointment
0,Pt aware that he needs ROV for refill,Patient is returning nurse call. He is unable ...,PRESCRIPTION,REFILL,No
1,Mom wants to know if the Focalin needs some do...,school teacher is reporting pt is not able to ...,ASK_A_DOCTOR,MEDICATION RELATED,No
2,pt called to discuss nortryptiline. she says s...,Prescription Prescription Comments pt called ...,ASK_A_DOCTOR,MEDICATION RELATED,No
3,FYI Nortryptline medication.,Prescription Prescription Comments pt called ...,MISCELLANEOUS,OTHERS,No
4,Letter of patient establishment request,Requesting a letter stating that pt is indeed ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No


# DATA EXPLORING 

In [7]:
#shape or size of data frame 
actual_data.shape

(57280, 5)

In [7]:
#no of missing values 
actual_data.apply(lambda x: sum(x.isnull()),axis=0)

summary           3371
data              2706
categories           0
sub_categories       0
appointment          2
dtype: int64

In [8]:
#Percentage of missing values in each column 
actual_data.apply(lambda x: sum(x.isnull())/len(actual_data),axis=0)

summary           0.058851
data              0.047242
categories        0.000000
sub_categories    0.000000
appointment       0.000035
dtype: float64

In [9]:
#data type of attributes 
actual_data.dtypes

summary           object
data              object
categories        object
sub_categories    object
appointment       object
dtype: object

In [10]:
#
actual_data.describe()

,summary,data,categories,sub_categories,appointment
count,53909,54574,57280,57280,57278
unique,41222,53210,8,22,5
top,Phone Note,Appointments Patient advised they may come int...,PRESCRIPTION,MEDICATION RELATED,No
freq,3143,107,15096,10598,41559


In [11]:
# number of levels in categories column
actual_data.categories.value_counts()

PRESCRIPTION     15096
APPOINTMENTS     13872
MISCELLANEOUS    12167
ASK_A_DOCTOR     11799
LAB               4321
JUNK                21
mISCELLANEOUS        3
asK_A_DOCTOR         1
Name: categories, dtype: int64

In [12]:
actual_data.sub_categories.value_counts()

MEDICATION RELATED                                 10598
NEW APPOINTMENT                                    10478
REFILL                                              9819
OTHERS                                              7356
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)     3550
LAB RESULTS                                         2650
PROVIDER                                            1972
QUERIES FROM PHARMACY                               1722
RESCHEDULING                                        1626
SHARING OF LAB RECORDS (FAX ; E-MAIL ; ETC.)        1425
PRIOR AUTHORIZATION                                 1226
SYMPTOMS                                            1201
CHANGE OF PROVIDER                                   953
RUNNING LATE TO APPOINTMENT                          694
CANCELLATION                                         663
QUERY ON CURRENT APPOINTMENT                         657
FOLLOW UP ON PREVIOUS REQUEST                        357
CHANGE OF HOSPITAL             

In [13]:
actual_data.appointment.value_counts()

No     41559
NO     15524
Yes      177
YES       17
yes        1
Name: appointment, dtype: int64

In [189]:
actual_data.categories.describe()

count            57280
unique               8
top       PRESCRIPTION
freq             15096
Name: categories, dtype: object

In [190]:
actual_data.sub_categories.describe()

count                  57280
unique                    22
top       MEDICATION RELATED
freq                   10598
Name: sub_categories, dtype: object

In [191]:
actual_data.data.describe()

count                                                 54574
unique                                                53210
top       Appointments Patient advised they may come int...
freq                                                    107
Name: data, dtype: object

In [192]:
actual_data.summary.describe()

count          53909
unique         41222
top       Phone Note
freq            3143
Name: summary, dtype: object

In [193]:
actual_data.summary.size

57280

In [194]:
actual_data.summary.shape

(57280,)

In [195]:
actual_data.appointment.describe()

count     57278
unique        5
top          No
freq      41559
Name: appointment, dtype: object

In [196]:
#Displaying first 10 rows of actual data 
actual_data.head(10)

,summary,data,categories,sub_categories,appointment
0,Pt aware that he needs ROV for refill,Patient is returning nurse call. He is unable ...,PRESCRIPTION,REFILL,No
1,Mom wants to know if the Focalin needs some do...,school teacher is reporting pt is not able to ...,ASK_A_DOCTOR,MEDICATION RELATED,No
2,pt called to discuss nortryptiline. she says s...,Prescription Prescription Comments pt called ...,ASK_A_DOCTOR,MEDICATION RELATED,No
3,FYI Nortryptline medication.,Prescription Prescription Comments pt called ...,MISCELLANEOUS,OTHERS,No
4,Letter of patient establishment request,Requesting a letter stating that pt is indeed ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
5,Appt question,Appointments Pt mom wants to know if pt should...,APPOINTMENTS,QUERY ON CURRENT APPOINTMENT,No
6,dizzy & double vision past 45 mins after CT,had CT this a. m. in Smithfield and is feeling...,ASK_A_DOCTOR,SYMPTOMS,No
7,Please refax neurocog order to new fac; Wake ...,Dr. is too far booked out per pt and like the ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
8,pt wants to reschedule Epidural from 04/30 ; p...,Appointments pt wants to reschedule Epidural f...,APPOINTMENTS,RESCHEDULING,No
9,Phone Note,NaN,MISCELLANEOUS,OTHERS,No


In [197]:
#Last 10 rows of actual data
actual_data.tail(10)

,summary,data,categories,sub_categories,appointment
57270,Pt questions if Dr. Gabr recommends she get th...,NaN,MISCELLANEOUS,OTHERS,No
57271,Please fax PT order to Holt PT fax #919-882-1727,Please fax PT order to Holt PT fax Faxed. P...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
57272,ADD ON w Holly for 05/13 @ 2:30 PM,Established Raleigh Neurology Assoc P A Starne...,APPOINTMENTS,NEW APPOINTMENT,Yes
57273,Namenda XR,Medfusion Secure Electronic Message Subject RE...,ASK_A_DOCTOR,MEDICATION RELATED,No
57274,pts wife would like a cb,pts wife would like a cb regarding portal mess...,PRESCRIPTION,REFILL,No
57275,Out of meds -LOST RXS,Prescription Refill ADDERALL XR MG q am ADDERA...,PRESCRIPTION,REFILL,No
57276,pt's mom called to report a migraine,Triage Pediatric Headache mom called and state...,ASK_A_DOCTOR,MEDICATION RELATED,No
57277,Dental clearance,RNA Dental office called wanting verbal clear...,MISCELLANEOUS,OTHERS,No
57278,Catheter documentation,CALL FROM OTHER CLINIC Supply Super Store Rea...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
57279,status of FMLA form?- awaiting MD completion,mom is calling to find out the status of her F...,MISCELLANEOUS,OTHERS,No


# Data Observartions 

We observer that our dat set having missing values,NaN values,not relavant data types, 
upper and lowercase letters in a sentences,punctuations,tags,numeric values, 
#Stop words, Spelling errors, non dictionary words


# To Over come all these we do preprocessing using NLTK

In [10]:
# Removing missing values
actual_data.dropna(axis = 0,how = 'any',inplace=True)
actual_data = actual_data.reset_index(drop=True)

In [198]:
# replacing na values with None word
#actual_data =actual_data.where((pd.notnull(actual_data)), None)

# replace na values with empty space
#actual_data= actual_data.replace(np.nan, '', regex=True)
#actual_data.head(10)

,summary,data,categories,sub_categories,appointment
0,Pt aware that he needs ROV for refill,Patient is returning nurse call. He is unable ...,PRESCRIPTION,REFILL,No
1,Mom wants to know if the Focalin needs some do...,school teacher is reporting pt is not able to ...,ASK_A_DOCTOR,MEDICATION RELATED,No
2,pt called to discuss nortryptiline. she says s...,Prescription Prescription Comments pt called ...,ASK_A_DOCTOR,MEDICATION RELATED,No
3,FYI Nortryptline medication.,Prescription Prescription Comments pt called ...,MISCELLANEOUS,OTHERS,No
4,Letter of patient establishment request,Requesting a letter stating that pt is indeed ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
5,Appt question,Appointments Pt mom wants to know if pt should...,APPOINTMENTS,QUERY ON CURRENT APPOINTMENT,No
6,dizzy & double vision past 45 mins after CT,had CT this a. m. in Smithfield and is feeling...,ASK_A_DOCTOR,SYMPTOMS,No
7,Please refax neurocog order to new fac; Wake ...,Dr. is too far booked out per pt and like the ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
8,pt wants to reschedule Epidural from 04/30 ; p...,Appointments pt wants to reschedule Epidural f...,APPOINTMENTS,RESCHEDULING,No
9,Phone Note,,MISCELLANEOUS,OTHERS,No


In [199]:

#Handling missing values
#actual_data.dropna(axis=0, how='any')

,summary,data,categories,sub_categories,appointment
0,Pt aware that he needs ROV for refill,Patient is returning nurse call. He is unable ...,PRESCRIPTION,REFILL,No
1,Mom wants to know if the Focalin needs some do...,school teacher is reporting pt is not able to ...,ASK_A_DOCTOR,MEDICATION RELATED,No
2,pt called to discuss nortryptiline. she says s...,Prescription Prescription Comments pt called ...,ASK_A_DOCTOR,MEDICATION RELATED,No
3,FYI Nortryptline medication.,Prescription Prescription Comments pt called ...,MISCELLANEOUS,OTHERS,No
4,Letter of patient establishment request,Requesting a letter stating that pt is indeed ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
5,Appt question,Appointments Pt mom wants to know if pt should...,APPOINTMENTS,QUERY ON CURRENT APPOINTMENT,No
6,dizzy & double vision past 45 mins after CT,had CT this a. m. in Smithfield and is feeling...,ASK_A_DOCTOR,SYMPTOMS,No
7,Please refax neurocog order to new fac; Wake ...,Dr. is too far booked out per pt and like the ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
8,pt wants to reschedule Epidural from 04/30 ; p...,Appointments pt wants to reschedule Epidural f...,APPOINTMENTS,RESCHEDULING,No
9,Phone Note,,MISCELLANEOUS,OTHERS,No


In [11]:
#no of missing values 
actual_data.apply(lambda x: sum(x.isnull()),axis=0)

summary           0
data              0
categories        0
sub_categories    0
appointment       0
dtype: int64

In [12]:
actual_data.shape

(51202, 5)

In [13]:
#Converting datatype of summary feature
actual_data.summary =actual_data.summary.astype('str')
#Converting datatype of data feature
actual_data.data =actual_data.data.astype('str')
#Converting datatype of category feature
actual_data.categories=actual_data.categories.astype("category")
#Converting datatype of sub_categories feature
actual_data.sub_categories=actual_data.sub_categories.astype("category")
#Converting datatype of appointment feature
actual_data.appointment=actual_data.appointment.astype("category")

In [203]:
#Converting to lower case

In [14]:
actual_data['summary'] = actual_data['summary'].astype(str).str.lower()
actual_data["data"] = actual_data["data"].astype(str).str.lower()
actual_data["categories"] = actual_data["categories"].map(lambda x: x.upper())
actual_data["sub_categories"] = actual_data["sub_categories"].map(lambda x: x.upper())
actual_data["appointment"] = actual_data["appointment"].map(lambda x: x.upper())

In [17]:
ax = sns.countplot(actual_data.categories,label="Count")       
PRESCRIPTION,APPOINTMENTS,MISCELLANEOUS,ASK_A_DOCTOR,LAB,JUNK = actual_data.categories.value_counts()
print('Number of PRESCRIPTION: ',PRESCRIPTION)
print('Number of APPOINTMENTS : ',APPOINTMENTS)
print('Number of MISCELLANEOUS  : ',MISCELLANEOUS )
print('Number of ASK_A_DOCTOR  : ',ASK_A_DOCTOR)
print('Number of LAB  : ',LAB )
print('Number of JUNK    : ',JUNK )
# represent size of the graph
from pylab import rcParams
rcParams['figure.figsize'] = 10,5

Number of PRESCRIPTION:  14442
Number of APPOINTMENTS :  11670
Number of MISCELLANEOUS  :  10924
Number of ASK_A_DOCTOR  :  9943
Number of LAB  :  4220
Number of JUNK    :  3


In [15]:
actual_data.sub_categories.value_counts()

MEDICATION RELATED                                 10480
REFILL                                              9626
NEW APPOINTMENT                                     7710
OTHERS                                              5302
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)     3425
LAB RESULTS                                         2612
QUERIES FROM PHARMACY                               1721
PROVIDER                                            1594
RESCHEDULING                                        1496
SHARING OF LAB RECORDS (FAX ; E-MAIL ; ETC.)        1381
SYMPTOMS                                            1190
PRIOR AUTHORIZATION                                 1152
CHANGE OF PROVIDER                                   915
RUNNING LATE TO APPOINTMENT                          694
QUERY ON CURRENT APPOINTMENT                         649
CANCELLATION                                         602
FOLLOW UP ON PREVIOUS REQUEST                        349
CHANGE OF HOSPITAL             

In [43]:
#sub_cat = sns.countplot(actual_data.sub_categories,label="Count")       
#MEDICATION RELATED,NEW APPOINTMENT,REFILL,OTHERS,HEALTH RECORDS,LAB RESULTS,PROVIDER,QUERIES FROM PHARMACY,RESCHEDULING,LAB RECORDS,PRIOR AUTHORIZATION,SYMPTOMS,CHANGE OF PROVIDER,RUNNING LATE TO APPOINTMENT,CANCELLATION,QUERY ON CURRENT APPOINTMENT,FOLLOW UP ON PREVIOUS REQUEST,CHANGE OF HOSPITAL,QUERIES FROM INSURANCE FIRM,CHANGE OF PHARMACY,JUNK = actual_data.sub_categories.value_counts()
sub_x = sns.countplot(actual_data.sub_categories,label="Count")       
#MED,NEW_APP,REFILL,OTHERS,HEALTH_RECORDS,LAB_RESULTS,PROVIDER,QUERIES_PHARM,RESCHED,PRIOR_AUTHOR,SYMPTOMS,CH_PROVIDER,LATE_APPT,CANCEL,QUERY_APPT,Fw_pr_req,Ch_hop,query_insur,ch_phar,junk = actual_data.sub_categories.value_counts()
# represent size of the graph
from pylab import rcParams
rcParams['figure.figsize'] = 40,8

In [16]:
actual_data.data[0]

'patient is returning nurse call. he is unable to make appt without talking to fin service dept. however he needs medication and worried that he will have issue without medication. please call patient to discuss.     pt returned phone call. please call back to advise   what is the problem? is he without insurance? he has been with instructions to come in for a appt. and cannot have refills without one.  david   rn spoke with pt and relayed the above to him.  he requested to speak with financial services.  rn transferred him to the business office.  rn requested business office to call once matter has been completed.  hollie saltis rn   ok.  david'

In [17]:
actual_data.data[100]

'prescription refill klonopin mg tabs po qhs pt states cvs did not get the rx. wants to know if it can be sent again. says she only has enough for nights.    rna  talked pt to inform rx being faxed this morning to pharmacy. asked to call pharmacy in next couple hrs or so to confirm receipt and if not received call rna back to inform. pt verbalized understanding.   tisha walker rn'

In [ ]:
##############################################################removing stop words

In [18]:
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def raw_to_prepwithtokenize( raw_review ):
    review_text = BeautifulSoup(raw_review).get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    low_case = letters_only.lower()
    words = nltk.word_tokenize(low_case)
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    processed_sentence = " ".join(meaningful_words)
    return(processed_sentence)
################################################################################################

In [19]:
num_reviews=len(actual_data)
num_reviews

51202

In [20]:
process=[]
for i in range (0,num_reviews):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    process.append(raw_to_prepwithtokenize(actual_data['data'][i]))

no.of rows reviews processed: 0


E:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file E:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000


E:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


no.of rows reviews processed: 50000


In [21]:
process_summary=[]
for i in range (0,num_reviews):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    process_summary.append(raw_to_prepwithtokenize(actual_data['summary'][i]))

no.of rows reviews processed: 0


E:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file E:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000
no.of rows reviews processed: 50000


In [22]:
process[0:2]

['patient returning nurse call unable make appt without talking fin service dept however needs medication worried issue without medication please call patient discuss pt returned phone call please call back advise problem without insurance instructions come appt refills without one david rn spoke pt relayed requested speak financial services rn transferred business office rn requested business office call matter completed hollie saltis rn ok david',
 'school teacher reporting pt able sit still mom wants know focalin needs dosage adjusting something pt could take school staff could administer please call back discuss mom sts patient alot issues meds effectiveness lov rescheduled show mom apologized show sts disconnected home number use cell appt scheduled advd phone call completed appt scheduled marcia richardson lpn']

In [24]:
process_summary[0:2]

['pt aware needs rov refill', 'mom wants know focalin needs dosage adjusting']

In [25]:
import re
def replacing_text(input_text):
    letters_only = re.sub("[^a-zA-Z]", " ",input_text )
    low_case = letters_only.lower()
    #words = nltk.word_tokenize(low_case)
    input_text   =re.sub(' pt ', ' patient ',  low_case)
    input_text1  =re.sub(' appt ', ' appointment ',  input_text)
    input_text2  =re.sub(' ov ', ' office visit ',  input_text1)
    input_text3  =re.sub(' schd ', ' scheduled ',  input_text2)
    input_text4  =re.sub(' np ', ' new patient ',  input_text3)
    input_text5  =re.sub(' r/s ', ' reschedule ',  input_text4)
    input_text6  =re.sub(' med ', ' medication ',  input_text5)
    input_text7  =re.sub(' pa ', ' prior authorization ',  input_text6)
    input_text8  =re.sub(' fin ', ' financial ',  input_text7)
    input_text9  =re.sub(' dept ', ' department ',  input_text8) 
    input_text10 =re.sub(' Call patient back ', ' ', input_text9)
    input_text11 =re.sub(' Phone Note ', ' ', input_text10)
    input_text12 =re.sub(' Caller name ', ' ', input_text11)
    input_text13 =re.sub(' Call Taken by ', ' ', input_text12)
    input_text14 =re.sub(' Cell Phone ', ' ', input_text13)
    input_text15 =re.sub(' CALL FROM PATIENT ', ' ', input_text14)
    input_text16 =re.sub(' CALL PATIENT BACK AT ', ' ', input_text15)
    input_text17 =re.sub(' Additional Follow-up Details ', ' ', input_text16)
    input_text18 =re.sub(' Follow-up Details ', ' ', input_text17)
    input_text19 =re.sub(' Additional Follow-up by ', ' ', input_text18)
    input_text20 =re.sub(' Follow-up by ', ' ', input_text19)
    input_text21 =re.sub(' Default Paragraph ', ' ', input_text20)
    input_text22 =re.sub(' Default Paragraph ', ' ', input_text21)
    input_text22 =re.sub(' n ', ' and ', input_text21)  
    return(input_text22)    

In [26]:
data_process2=[]
for i in range (0,num_reviews):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    data_process2.append(replacing_text(process[i]))

no.of rows reviews processed: 0
no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000
no.of rows reviews processed: 50000


In [27]:
summary_process2=[]
for i in range (0,num_reviews):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    summary_process2.append(replacing_text(process_summary[i]))

no.of rows reviews processed: 0
no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000
no.of rows reviews processed: 50000


In [28]:
data_process2[0:2]

['patient returning nurse call unable make appointment without talking financial service department however needs medication worried issue without medication please call patient discuss patient returned phone call please call back advise problem without insurance instructions come appointment refills without one david rn spoke patient relayed requested speak financial services rn transferred business office rn requested business office call matter completed hollie saltis rn ok david',
 'school teacher reporting patient able sit still mom wants know focalin needs dosage adjusting something patient could take school staff could administer please call back discuss mom sts patient alot issues meds effectiveness lov rescheduled show mom apologized show sts disconnected home number use cell appointment scheduled advd phone call completed appointment scheduled marcia richardson lpn']

In [30]:
summary_process2[0:2]

['pt aware needs rov refill', 'mom wants know focalin needs dosage adjusting']

In [31]:
actual_data.summary=pd.DataFrame(summary_process2)
actual_data.data=pd.DataFrame(data_process2)

In [45]:
actual_data.head(2)

,summary,data,categories,sub_categories,appointment
0,pt aware needs rov refill,patient returning nurse call unable make appoi...,PRESCRIPTION,REFILL,NO
1,mom wants know focalin needs dosage adjusting,school teacher reporting patient able sit stil...,ASK_A_DOCTOR,MEDICATION RELATED,NO


In [46]:
actual_data.dtypes

summary             object
data                object
categories        category
sub_categories    category
appointment       category
dtype: object

In [47]:
actual_data.data[0]

'patient returning nurse call unable make appointment without talking financial service department however needs medication worried issue without medication please call patient discuss patient returned phone call please call back advise problem without insurance instructions come appointment refills without one david rn spoke patient relayed requested speak financial services rn transferred business office rn requested business office call matter completed hollie saltis rn ok david'

In [50]:
actual_data.dtypes

summary             object
data                object
categories        category
sub_categories    category
appointment       category
dtype: object

In [32]:
#Converting datatype of summary feature
actual_data.summary =actual_data.summary.astype('str')
#Converting datatype of data feature
actual_data.data =actual_data.data.astype('str')
#Converting datatype of category feature
actual_data.categories=actual_data.categories.astype("category")
#Converting datatype of sub_categories feature
actual_data.sub_categories=actual_data.sub_categories.astype("category")
#Converting datatype of appointment feature
actual_data.appointment=actual_data.appointment.astype("category")

In [51]:
actual_data.head(5)

,summary,data,categories,sub_categories,appointment
0,pt aware needs rov refill,patient returning nurse call unable make appoi...,PRESCRIPTION,REFILL,NO
1,mom wants know focalin needs dosage adjusting,school teacher reporting patient able sit stil...,ASK_A_DOCTOR,MEDICATION RELATED,NO
2,pt called discuss nortryptiline says weird tas,prescription prescription comments patient cal...,ASK_A_DOCTOR,MEDICATION RELATED,NO
3,fyi nortryptline medication,prescription prescription comments patient cal...,MISCELLANEOUS,OTHERS,NO
4,letter patient establishment request,requesting letter stating patient indeed patie...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),NO


In [39]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features= 5000)
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=5,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [40]:
data_feaures = vect.fit_transform(actual_data['data'])

In [41]:
summary_features = vect.fit_transform(actual_data['summary'])

dtype('int64')

In [42]:
data_features=data_feaures.toarray()
summary_features = summary_features.toarray()

In [43]:
features = np.concatenate((data_features,summary_features), axis=0)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [62]:
preprocessed_data=pd.concat([df_data,df_summary],axis=1)

MemoryError: 

In [66]:
a = np.array([1,2,3])
b = np.array([4,5,6])
np.concatenate((a,b), axis=0)

array([1, 2, 3, 4, 5, 6])

In [52]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
#to find the frequency of each word count


In [55]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(actual_data.data)
tokenizer.fit_on_texts(actual_data.summary)

train_data_x = tokenizer.texts_to_sequences(actual_data.data)
train_data_x = pad_sequences(train_data_x, maxlen=50)
train_summary_x = tokenizer.texts_to_sequences(actual_data.summary)
train_summary_x = pad_sequences(train_summary_x, maxlen=50)

In [56]:
print(train_data_x.shape, train_summary_x.shape)

(51202, 50) (51202, 50)


In [57]:
train_data_x[1:3]

array([[ 138,   12,   61,   38,  468,   36,  575, 3062,  389,    1,  131,
          49,  196, 1559,  131, 2806,    9,    2,   16,   63,   12,  481,
           1, 1362,  328,  126, 4474,  186,  741,  704,   12, 2354,  704,
         481, 2105,  127,  187,  300,  378,   11,   50, 1009,   13,    2,
          29,   11,   50,  171,  167,   56],
       [   0,    0,    0,    0,    0,    0,    0,    0,   22,   22,  191,
           1,    6,   63, 6081,   67, 3249, 3958,  556,    1,   80, 4679,
        7866,    9,    2,   63,    7,    1,   25, 1025, 6622,   37,   97,
         230,  760,  524, 1835, 3286,   40, 8517,    9,   28,   94,  107,
         235,   56,    7,  101,  354,   14]])

In [64]:
vocab = tokenizer.word_counts
vocab

OrderedDict([('patient', 104720),
             ('returning', 1168),
             ('nurse', 2514),
             ('call', 54390),
             ('unable', 1692),
             ('make', 2440),
             ('appointment', 22172),
             ('without', 898),
             ('talking', 226),
             ('financial', 241),
             ('service', 217),
             ('department', 535),
             ('however', 504),
             ('needs', 9914),
             ('medication', 18788),
             ('worried', 189),
             ('issue', 670),
             ('please', 26969),
             ('discuss', 7115),
             ('returned', 1822),
             ('phone', 19168),
             ('back', 17338),
             ('advise', 12746),
             ('problem', 862),
             ('insurance', 2685),
             ('instructions', 6707),
             ('come', 3436),
             ('refills', 1916),
             ('one', 5664),
             ('david', 478),
             ('rn', 40311),
             ('spoke

In [65]:
train_summary_x=pd.DataFrame(train_summary_x)
train_data_x=pd.DataFrame(train_data_x)
preprocessed_data=pd.concat([train_data_x,train_summary_x],axis=1)

In [66]:
train_x=preprocessed_data

In [91]:
unique_labels = list(actual_data.categories.unique())
train_y = np.array([unique_labels.index(i) for i in actual_data.categories])
train_y = to_categorical(train_y)

In [95]:
print(train_y.shape)
print(train_x.shape)

(51202, 6)
(51202, 100)


In [99]:
m_train_y = actual_data['categories']

In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, m_train_y, test_size=0.20, random_state=42)

In [ ]:
#applying random forest

In [107]:
from sklearn.ensemble import RandomForestClassifier


In [125]:
model_random = RandomForestClassifier(n_estimators = 5000)

In [127]:
model_fit = model_random.fit(X_train,y_train)

In [122]:
pred_test = model_fit.predict(X_test)

In [123]:
from sklearn.metrics import accuracy_score,classification_report,precision_score,recall_score

In [117]:
print(recall_score(y_test,pred_test))

E:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'               precision    recall  f1-score   support\n\n APPOINTMENTS       0.63      0.58      0.61      2134\n ASK_A_DOCTOR       0.46      0.60      0.52      2349\n         JUNK       0.00      0.00      0.00         1\n          LAB       0.88      0.24      0.38       844\nMISCELLANEOUS       0.53      0.44      0.48      2060\n PRESCRIPTION       0.60      0.69      0.64      2853\n\n  avg / total       0.58      0.56      0.55     10241\n'

In [124]:
accuracy_score(y_test,pred_test)

0.56058978615369592

In [96]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers import Dense,Activation

model = Sequential()
model.add(Embedding(100000,128))
model.add(LSTM(200,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=False))
model.add(Dense(6))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [97]:
train_y

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.]])

In [ ]:
train_x.head()

In [98]:
batch_size=64
model.fit(train_x,train_y,batch_size=batch_size,epochs=5)

Epoch 1/5


KeyError: '[15416 45716 12573 14938 20349 42535 20607 26102 28014 39860 24373 45198\n 30548 12385 32013 20088 29564    57  6692  4641 32177  2152 20263 13294\n 19607 22019  5395 50476 19572 10190 25639 31996  1323 27379 36337 47392\n 11201 22931 20720 33801 23222 30781 41955 24589 45588  4665 49598   870\n 32904 36908 10885  2844  2459 13234 37054  3029 20944 35543 19665 18924\n  5423 21464 33497 17099] not in index'

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.20, random_state=42)

In [ ]:
test_predictions = model.predict_classes(X_test)

In [ ]:
list = []
list1=[]
for i in test_predictions:
    print(unique_labels[i])
    list.append(unique_labels[i])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
pre=pd.DataFrame(list)


In [ ]:
#accuracy_score(pre,y_test)

In [ ]:
%debug

In [ ]:
#Random forest
from sklearn.ensemble import RandomForestClassifier
rf_cla= RandomForestClassifier()
rf_cla.fit(train_x,train_y)

In [ ]:
predictions = rf_cla.predict(X_test)
print("train accuracy is:",accuracy_score(y_train,(rf_cla.predict(X_train))))
print("test accuracy is:",accuracy_score(y_test, predictions))

In [ ]:
test_predictions = model.predict_classes(X_test)